In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

C:\Users\swdtk\AppData\Local\Temp\ipykernel_8320\2721483381.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [81]:
def getRateDataFrame(codeStr):
    """
    関数の説明：
        codeStrで指定したレート一覧を、ヤフーファイナンスから取得する関数
    
    引数：
        codeStr:ヤフーファイナンスから取得する通貨名を指定

    戻値：
        rateDf:OHCLが記載されたDataFrame
    """    
    #ヤフーファイナンスからレートを取得
    rateDf = yf.download(codeStr, start='2004-01-01', end='2040-12-31')

    #不要な列の削除
    del rateDf['Adj Close']
    del rateDf['Volume']

    #indexを列に変換
    rateDf.reset_index(inplace= True)

    return rateDf


def makeHendoRitu(rateDf, nRaw):
    """
    関数の説明：
        n行前の始値と、現在の始値の変動率を記載する
    
    引数：
        rateDf:Date、Open、High、Low、Closeで構成されるレートの一覧
        nRaw:どの期間の変動率を取得するか（1なら1行前）

    戻値：
        rateDf:変動率を記載したDataFrame
    """
    #n行前の始値と、現在の始値の変動率を記載
    nDaysStr = str(nRaw) + 'period'
    rateDf[nDaysStr] = rateDf['Open'].shift(nRaw)
    rateDf['変動率'] = (( rateDf['Open'] / rateDf[nDaysStr] ) -1 ) * 100
    del rateDf[nDaysStr]

    return rateDf

def makePeriodDataFrame(nRaw):
    """
    関数の説明：
        各通貨の指定した期間の変動率を１つのDataFrameにまとめる関数
    
    引数：
        nRaw:指定した期間

    戻値：
        merge_Df:各通貨の指定した期間の変動率をまとめたDataFrame
    """
    merge_Df = pd.DataFrame()

    for codeKey in currencyList:
        #レート取得
        codeValue = getRateCodeDic[codeKey]
        rateDf = getRateDataFrame(codeStr=codeValue)

        #各通貨の指定した期間のDataFrameを取得する
        rateDf = makeHendoRitu(rateDf, nRaw=nRaw)
        rateDf = rateDf[['Date', '変動率']] #不要な列を削除
        rateDf = rateDf.rename(columns={'変動率':codeKey}) #列名を通貨名に変更

        #取得してきた各通貨の変動率をマージしていく
        if len(merge_Df) == 0:
            merge_Df = rateDf
        else:
            merge_Df = pd.merge(merge_Df, rateDf, how='left', on='Date')
    
    return merge_Df

In [61]:
#取得する通貨
currencyList = ['USD', 'EUR', 'GBP', 'NZD', 'AUD']

#取得してくるコード一覧（すべてクロス円）
getRateCodeDic = {'USD':'USDJPY=X','EUR':'EURJPY=X', 'GBP':'GBPJPY=X', 'NZD':'NZDJPY=X', 'AUD':'AUDJPY=X'}


In [82]:
part1Df = makePeriodDataFrame(nRaw=10)

c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd

In [83]:
part1Df

,Date,USD,EUR,GBP,NZD,AUD
0,2004-01-01,NaN,NaN,NaN,NaN,NaN
1,2004-01-02,NaN,NaN,NaN,NaN,NaN
2,2004-01-05,NaN,NaN,NaN,NaN,NaN
3,2004-01-06,NaN,NaN,NaN,NaN,NaN
4,2004-01-07,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
5215,2024-02-07,-0.292082,-1.216539,-0.981262,-0.344885,-1.151711
5216,2024-02-08,0.335383,-0.633640,-0.375096,0.365027,-0.482305
5217,2024-02-09,1.039781,0.391339,0.344688,1.024122,-0.260187
5218,2024-02-12,0.693269,0.280785,0.219057,1.603473,-0.120077


c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
c:\Users\swdtk\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd

In [80]:
merge_Df

,Date,USD,EUR,GBP,NZD,AUD
0,2004-01-01,NaN,NaN,NaN,NaN,NaN
1,2004-01-02,0.345959,-0.125775,-0.239541,-0.303770,0.000000
2,2004-01-05,-0.363399,0.081488,0.198355,0.726147,0.956007
3,2004-01-06,-0.645284,-0.236867,0.031257,0.616322,0.567480
4,2004-01-07,-0.018822,0.259687,0.703092,0.064622,0.147207
...,...,...,...,...,...,...
5215,2024-02-07,-0.553018,-0.414023,-0.011270,0.215656,0.157811
5216,2024-02-08,0.181308,0.365704,0.377923,0.341644,0.098477
5217,2024-02-09,0.792110,0.798108,0.732689,0.760561,0.433903
5218,2024-02-12,-0.061635,0.138020,0.073267,0.593541,0.350570


In [33]:
def makeSma(dataDf, tankiSma, choukiSma):
    """
    関数の説明：
        yfinanceで取得してきた株価のDataFrameを基に、
        ゴールデンクロスなら買い、デッドクロスなら決済のフラグを作成してくれる関数。
    
    Args:
        dataDf: yfinanceで取得してきた株価のDataFrame
        tankiSma: 短期移動平均線の日数
        choukiSma: 長期移動平均線の日数

    Returns:
        tradesDf: 売買フラグを付与したDataFrameを返す。
    """
    # 短期移動平均線を計算
    dataDf['短期SMA'] = dataDf['Adj Close'].rolling(window=tankiSma).mean()

    # 長期移動平均線を計算
    dataDf['長期SMA'] = dataDf['Adj Close'].rolling(window=choukiSma).mean()

    # ポジションを保持するための変数
    position = 0

    #ポジションをとった時のPrice
    numPositon = 0    

    # 取引履歴を保存するDataFrame
    tradesDf = pd.DataFrame(columns=['Date', 'Position', 'Price'])

    # 移動平均線を用いたバックテストを実施
    for i in range(len(dataDf)):
        # 短期移動平均線が長期移動平均線を上抜けした場合、買いポジションを取る
        if dataDf['短期SMA'][i] > dataDf['長期SMA'][i] and position == 0:
            position = 1
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)

        # 短期移動平均線が長期移動平均線を下抜けした場合、決済
        elif dataDf['短期SMA'][i] < dataDf['長期SMA'][i] and position == 1:
            position = 0
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
    
    #決済金額を追記
    tradesDf['result']=0

    for i in range(len(tradesDf)):
        #Buyなら買う
        if tradesDf.iloc[i, 1] == 'Buy':
            numPositon = tradesDf.iloc[i, 2]
        #Sellなら決済
        if tradesDf.iloc[i, 1] == 'Sell':
            tradesDf.iloc[i, 3] = (tradesDf.iloc[i, 2] - numPositon) / numPositon

    return tradesDf



In [6]:
n225_sma = makeSma(dataDf=n225_train, tankiSma=5, choukiSma=25)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\App

          Date Position         Price    result
0   2000-02-08      Buy  19868.880859  0.000000
1   2000-03-13     Sell  19189.929688 -0.034172
2   2000-03-24      Buy  19958.080078  0.000000
3   2000-04-18     Sell  18969.519531 -0.049532
4   2000-06-08      Buy  17004.339844  0.000000
..         ...      ...           ...       ...
235 2018-10-12     Sell  22694.660156 -0.005550
236 2018-11-12      Buy  22269.880859  0.000000
237 2018-11-15     Sell  21803.619141 -0.020937
238 2018-11-29      Buy  22262.599609  0.000000
239 2018-12-07     Sell  21678.679688 -0.026229

[240 rows x 4 columns]


In [7]:
n225_sma['result'].sum()

0.5362064095993865

In [12]:
columns = ['短期SMA', '長期SMA', '結果']
resultDf = pd.DataFrame(columns=columns)

In [35]:
resultDf.to_csv('result3.csv', index=False, encoding='cp932')



In [40]:
import warnings
warnings.filterwarnings('ignore')

#短期線
for i in range(51, 250):

    #長期線    
    for j in range(i+1, 300):
        
        n225_sma = makeSma(dataDf=n225_train, tankiSma=i, choukiSma=j)

        # 結果を保存
        new_row = {'短期SMA': i, '長期SMA': j, '結果': n225_sma['result'].sum()}
        resultDf = resultDf.append(new_row, ignore_index=True)
        
resultDf.to_csv('result.csv', index=False, encoding='cp932')

In [36]:
# 日経225のデータをyfinanceから取得
n225_test = yf.download('^N225', start='2019-01-01', end='2023-12-31')

# テストデータで検証
n225_sma = makeSma(dataDf=n225_test, tankiSma=195, choukiSma=296)

[*********************100%***********************]  1 of 1 completed


In [37]:
n225_sma['result'].sum()

0.2498771142944017

In [26]:
n225_sma.to_csv('test.csv')